In [5]:
import itertools
import random

In [6]:
spells = {
    "magic_missile": {
        "mana_cost": 53,
        "damage": 4,
    },
    "drain": {
        "mana_cost": 73,
        "damage": 2,
        "heals": 2
    },
    "shield": {
        "mana_cost": 113,
        "armor": 8,
        "lasts": 6
    },
    "poison": {
        "mana_cost": 173,
        "damage": 3,
        "lasts": 6
    },
    "recharge": {
        "mana_cost": 229,
        "mana_gain": 101,
        "lasts": 5
    },
}

In [7]:
day22_inputs = "inputs/day22.txt"
with open(day22_inputs, 'r') as file:
    day22_data = file.readlines()
day22_data_split = [el.strip().split(": ") for el in day22_data]
day22_data_flat = list(itertools.chain.from_iterable(day22_data_split))

boss = {day22_data_flat[i]: int(day22_data_flat[i + 1]) for i in range(0, len(day22_data_flat), 2)}

print(boss)

{'Hit Points': 55, 'Damage': 8}


In [150]:
def player_effects(player, active_effects, curr_player_mana_points, curr_player_armor, curr_boss_hit_points):
    active_effects_keys = list(active_effects.keys())
    print(active_effects)
    
    for effect in active_effects_keys:
        turns_left = active_effects[effect]
        new_turns_left = turns_left - 1
        player["Effects"][effect] = new_turns_left
        if effect == "recharge":
            curr_player_mana_points += spells[effect]["mana_gain"]
            print(f"{effect} provides mana, player has {curr_player_mana_points} mana points; effect timer is now {new_turns_left}")
        if effect == "poison":
            curr_boss_hit_points -= spells[effect]["damage"]
            print(f"{effect} deals damage, boss has {curr_boss_hit_points}; effect timer is now {new_turns_left}")
        if new_turns_left == 0:
            if effect == "shield":
                curr_player_armor = 0
            player["Effects"].pop(effect)
            print(f"{effect} has worn off")
    return {
        "player_mana": curr_player_mana_points,
        "player_armor": curr_player_armor,
        "boss_hp": curr_boss_hit_points
    }

def player_turn(player, spells, spell, curr_boss_hit_points, curr_player_mana_points, curr_player_armor, curr_player_hit_points, curr_turn_player_damage):
    spell_mana_costs = spells[spell]["mana_cost"]
            
    if spell_mana_costs < curr_player_mana_points:
        curr_player_mana_points -= spell_mana_costs
        print(f"player casts {spell} for {spell_mana_costs} and has {curr_player_mana_points} mana remaining")
        try: 
            spell_lasts = spells[spell]["lasts"]
            player["Effects"].setdefault(spell, spell_lasts)
            
            if spell == "shield":
                curr_player_armor = 7
        except: 
            # if not a spell with an effect, cast normally
            curr_turn_player_damage += spells[spell]["damage"]
            curr_boss_hit_points -= curr_turn_player_damage
            print(f"boss takes {curr_turn_player_damage} of damage and has {curr_boss_hit_points} hit points")
            
            if spell == "drain": 
                curr_player_hit_points += spells[spell]["heals"]
                print(f"player heals and has {curr_player_hit_points} hit ponts")
    return {
        "player_hp": curr_player_hit_points,
        "player_mana": curr_player_mana_points,
        "player_armor": curr_player_armor,
        "boss_hp": curr_boss_hit_points
    }

In [151]:
def generate_attack_list(spells, boss, curr_player_hit_points):
    # how many boss attacks before player dies if default player health
    boss_attacks = -(curr_player_hit_points // -boss["Damage"])
    print(f"with the player's default health of {curr_player_hit_points} and boss attack of {boss["Damage"]}, player would die in {boss_attacks} turns")

    # spells damage per mana
    for spell in spells:
        try:
            spell_damage = spells[spell]["damage"]
            mana_cost = spells[spell]["mana_cost"]
            
            damage_per_mana = spell_damage / mana_cost
            print(spell, damage_per_mana)
        except:
            pass

    print("")
    
    # create a small simulation
    
    # stage 1: cast as many magic_missiles before player health is too low
    turns_before_death = (boss_attacks - 1)
    spells_to_cast = ["magic_missile"] * turns_before_death
    player_health_left = curr_player_hit_points - (boss["Damage"]*turns_before_death)
    print(f"at stage one, after {turns_before_death} ")
    
    # stage 2: cast drain to heal
    spells_to_cast += ["drain"] * 2
    
    
    return spells_to_cast

In [152]:
player = {
    "Hit Points": 50,
    "Mana Points": 500,
    "Damage": 0,
    "Armor": 0,
    "Effects": {}
}

curr_boss_hit_points = boss["Hit Points"]

curr_player_hit_points = player["Hit Points"]
curr_player_mana_points = player["Mana Points"]

curr_player_armor = player["Armor"]
curr_player_damage = player["Damage"]

spells_to_cast = generate_attack_list(spells, boss, curr_player_hit_points)
curr_spell_i = 0
print(spells_to_cast)

in_play = True
while in_play:
    for turn in ["player", "boss"]:
        print(f"--- {turn} turn ---")

        active_effects = player["Effects"]
        result = player_effects(player, active_effects, curr_player_mana_points, curr_player_armor, curr_boss_hit_points)
        
        curr_player_mana_points = result['player_mana']
        curr_player_armor = result['player_armor']
        curr_boss_hit_points = result['boss_hp']
        
        print(f"player: {curr_player_hit_points} hit points, {curr_player_armor} armor, {curr_player_mana_points} mana points")
        print(f"boss: {curr_boss_hit_points} hit points")
        
        if turn == "player":
            curr_turn_player_damage = 0

            # spell_choice = random.choice([spell for spell in spells if spell not in active_effects])
            spell_choice = spells_to_cast[curr_spell_i]
            
            result = player_turn(player, spells, spell_choice, curr_boss_hit_points, curr_player_mana_points, curr_player_armor, curr_player_hit_points, curr_turn_player_damage)
            
            curr_player_hit_points = result['player_hp']
            curr_player_mana_points = result['player_mana']
            curr_player_armor = result['player_armor']
            curr_boss_hit_points = result['boss_hp']
            print(curr_spell_i)
            curr_spell_i += 1
        elif turn == "boss":
            boss_damage = max(1, boss["Damage"] - player["Armor"])
            curr_player_hit_points -= boss_damage  
            print(f"boss deals {boss_damage} to player, player has {curr_player_hit_points} hit points")
        
        if curr_boss_hit_points <= 0:
            print("boss has died")
            in_play = False
        if curr_player_hit_points <= 0:
            print("player has died")
            in_play = False 
        
        print("")

with the player's default health of 50 and boss attack of 8, player would die in 7 turns
magic_missile 0.07547169811320754
drain 0.0273972602739726
poison 0.017341040462427744

at stage one, after 6 
['magic_missile', 'magic_missile', 'magic_missile', 'magic_missile', 'magic_missile', 'magic_missile', 'drain', 'drain']
--- player turn ---
{}
player: 50 hit points, 0 armor, 500 mana points
boss: 55 hit points
player casts magic_missile for 53 and has 447 mana remaining
boss takes 4 of damage and has 51 hit points
0

--- boss turn ---
{}
player: 50 hit points, 0 armor, 447 mana points
boss: 51 hit points
boss deals 8 to player, player has 42 hit points

--- player turn ---
{}
player: 42 hit points, 0 armor, 447 mana points
boss: 51 hit points
player casts magic_missile for 53 and has 394 mana remaining
boss takes 4 of damage and has 47 hit points
1

--- boss turn ---
{}
player: 42 hit points, 0 armor, 394 mana points
boss: 47 hit points
boss deals 8 to player, player has 34 hit points

-